# Borrowing Hydrogen Calculations

In [1]:
# Setup
from pint import UnitRegistry
units = UnitRegistry()
units.load_definitions("units.txt")
Q_ = units.Quantity 
from cheminventory import ChemInventory, Location
from dotenv import load_dotenv
import os
import pandas as pd

## Data

In [6]:
data = [
    {
        'compound': 'toluene',
        'tags': ['solvent'],
        'unit_price': 43.30*units.GBP/(1*units.liter),
        "units": 5*units.liter,
        'amount_per_rxn': 1*units.milliliter,
        "boiling_point": Q_(110, units.degC),
        "melting_point": Q_(-93, units.degC),
        "cas_number": "108-88-3",
        "smiles": "Cc1ccccc1",
        'url': 'https://www.sigmaaldrich.com/catalog/product/sial/244511?lang=en&region=GB'
    },
    {
        'compound': '1-Z-Piperazine',
        'tags': ['starting_material', "amine"],
        'unit_price': 193.00*units.GBP/(25.0*units.ml),
        "units": 25*units.ml,
        "boiling_point": Q_(158, units.degC),
        "density": 1.142*units.gram/(1*units.ml),
        "cas_number": "31166-44-6",
        "smiles": "O=C(OCc1ccccc1)N2CCNCC2",
        'url': 'https://www.sigmaaldrich.com/catalog/product/aldrich/456926?lang=en&region=GB'
    },
    {
        "compound": "cyclobutanol",
        "tags": ["starting_material", "alcohol"],
        "unit_price": (283*units.GBP)/(5*units.gram),
        "units": 10*units.gram,
        "boiling_point": Q_(123, units.degC),
        "density": 0.921*units.gram/(1*units.milliliter),
        "melting_point": Q_(25.93, units.degC),
        "smiles": "OC1CCC1",
        "cas_number": "2919-23-5",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/156434?lang=en&region=GB"
    },
    {
        "compound": "cyclopentaol",
        "tags": ["starting_material", "alcohol"],
        "unit_price": 66.30*units.GBP/(500*units.milliliter),
        "units": 500*units.ml,
        "boiling_point": Q_(139, units.degC),
        "melting_point": Q_(-19.0, units.degC),
        "cas_number": "96-41-3",
        "smiles": "OC1CCC1",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/c112208?lang=en&region=GB"
    },
    {
        "compound": "cyclohexanol",
        "tags": ["starting_material", "alcohol"],
        "unit_price": 24.70*units.GBP/(1000*units.ml),
        "units": 1000*units.ml,
        "boiling_point": Q_(161.8, units.degC),
        "melting_point": Q_(25.93, units.degC),
        "cas_number": "96-41-3",
        "smiles": "OC1CCCC1",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/c112208?lang=en&region=GB"
    },
    {
        "compound": "cycloheptanol",
        "tags": ["starting_material", "alcohol"],
        "unit_price": 17.70*units.GBP/(25*units.gram),
        "units": 25*units.gram,
        "melting_point": Q_(185, units.degC),
        "density": 0.948*units.gram/(1*units.ml),
        "boiling_point": Q_(161.8, units.degC),
        "smiles": "OC1CCCCCC1",
        "cas_number": "502-41-0",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/c98802?lang=en&region=GB"
    },
    {
        "compound": "Dichloro(p-cymene)ruthenium(II) dimer",
        "tags": ["catalyst"],
        "unit_price": 170.00*units.GBP/(5*units.gram),
        "units": 60*units.gram,
        "cas_number": "52462-29-0",
        "url": ""
    },
    {
        "compound": "DPEphos",
        "tags": ["ligand"],
        "unit_price": (151.0*units.GBP)/(25*units.gram),
        "units": 50*units.gram,
        "cas_number": "166330-10-5",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/510017?lang=en&region=GB"
    },
    {
        "compound": "DPPF",
        "tags": ["ligand"],
        "unit_price": (409*units.GBP)/(25*units.gram),
        "units": 50*units.gram,
        "cas_number": "12150-46-8",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/510017?lang=en&region=GB"
    },

]

In [12]:
# Irridium catalyst is super expensive
ir = {
        "compound": "Pentamethylcyclopentadienyliridium(III) chloride,dimer",
        "tags": ["catalyst"],
        "unit_price": 261.00*units.GBP/(500*units.mg),
        "units": 500*units.gram,
        "cas_number": "12354-84-6",
        "url": "https://www.sigmaaldrich.com/catalog/product/aldrich/357537?lang=en&region=GB"
    }

## Calculations

In [4]:
def total_price(row):
    return row["unit_price"]*row["units"]

class CheckCI:
    def __init__(self):
        #Loads email and password from .env file
        load_dotenv()
        self.ci = ChemInventory()

    def __call__(self, row):
        cas_number = row["cas_number"]
        units_required = row["units"]
        results = self.ci.search(cas_number)
        if results:
            sizes = [units(res.size) for res in results]
            if sum(sizes) >= units_required:
                return True
        return False

In [7]:
df = pd.DataFrame(data)

# Check if the compound is already stocked in the lab
checkci = CheckCI()
df["stocked_in_lab"] = df.apply(checkci, axis=1)

# Calculate the number of reactions that could be run with each starting material

# Get the total price
total = df[df["stocked_in_lab"] == False].apply(total_price, axis=1).sum()
print("Total price: ", total)
df

Total price:  4027.7 GBP


/Users/Kobi/Library/Caches/pypoetry/virtualenvs/multitask-k0YbXS9f-py3.7/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:1638: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


,compound,tags,unit_price,units,amount_per_rxn,boiling_point,melting_point,cas_number,smiles,url,density,stocked_in_lab
0,toluene,[solvent],43.3 GBP / liter,5 liter,1 milliliter,110 degree_Celsius,-93 degree_Celsius,108-88-3,Cc1ccccc1,https://www.sigmaaldrich.com/catalog/product/s...,NaN,True
1,1-Z-Piperazine,"[starting_material, amine]",7.72 GBP / milliliter,25 milliliter,NaN,158 degree_Celsius,NaN,31166-44-6,O=C(OCc1ccccc1)N2CCNCC2,https://www.sigmaaldrich.com/catalog/product/a...,1.142 gram / milliliter,False
2,cyclobutanol,"[starting_material, alcohol]",56.6 GBP / gram,10 gram,NaN,123 degree_Celsius,25.93 degree_Celsius,2919-23-5,OC1CCC1,https://www.sigmaaldrich.com/catalog/product/a...,0.921 gram / milliliter,False
3,cyclopentaol,"[starting_material, alcohol]",0.1326 GBP / milliliter,500 milliliter,NaN,139 degree_Celsius,-19.0 degree_Celsius,96-41-3,OC1CCC1,https://www.sigmaaldrich.com/catalog/product/a...,NaN,False
4,cyclohexanol,"[starting_material, alcohol]",0.0247 GBP / milliliter,1000 milliliter,NaN,161.8 degree_Celsius,25.93 degree_Celsius,96-41-3,OC1CCCC1,https://www.sigmaaldrich.com/catalog/product/a...,NaN,False
5,cycloheptanol,"[starting_material, alcohol]",0.708 GBP / gram,25 gram,NaN,161.8 degree_Celsius,185 degree_Celsius,502-41-0,OC1CCCCCC1,https://www.sigmaaldrich.com/catalog/product/a...,0.948 gram / milliliter,False
6,Dichloro(p-cymene)ruthenium(II) dimer,[catalyst],34.0 GBP / gram,60 gram,NaN,NaN,NaN,52462-29-0,NaN,,NaN,False
7,DPEphos,[ligand],6.04 GBP / gram,50 gram,NaN,NaN,NaN,166330-10-5,NaN,https://www.sigmaaldrich.com/catalog/product/a...,NaN,False
8,DPPF,[ligand],16.36 GBP / gram,50 gram,NaN,NaN,NaN,12150-46-8,NaN,https://www.sigmaaldrich.com/catalog/product/a...,NaN,False


In [50]:
molar_mass = 612.39*units.mg/units.mmol
conc_cat = 383*units.mg/molar_mass/(25*units.ml)
flow_rate = 8.0*units.ml/units.min
consumption_rate = conc_cat*flow_rate
consumption_rate_mass = consumption_rate*molar_mass
stock_time = df.iloc[6]["units"]/consumption_rate_mass
print("Est. number of hours can run with catalyst stock: ", stock_time.to("hours").round(1))

Est. number of hours can run with catalyst stock:  8.2 hour
